# Fragstats Layers Reprojection and Organization

## Metadata
- **Script Name**: Reproject Fragsats Layers.ipynb
- **Project**: EarthCul - Cultural Ecosystem Services Analysis
- **Module**: Landscape Variables Processing
- **Author**: Carlos Javier Navarro
- **Date Created**: 2024
- **Version**: 1.0

## Description
This notebook handles the post-processing of landscape metrics calculated by Fragstats software. It performs coordinate system transformations and file organization to prepare landscape variables for integration with other environmental datasets in the EarthCul project.

## Objectives
1. **Reproject Fragstats Output**: Transform landscape metrics from original projection to standardized coordinate systems
2. **File Organization**: Move and organize landscape metric files into proper directory structures
3. **Quality Control**: Ensure consistent spatial properties across all landscape variables
4. **Batch Processing**: Handle multiple parks and metrics efficiently

## Study Areas
Processes landscape metrics for 8 Spanish National Parks:
- Aigüestortes i Estany de Sant Maurici (Aiguestortes)
- Ordesa y Monte Perdido (Ordesa)
- Peneda-Gerês (Peneda)
- Sierra de Guadarrama (Guadarrama)
- Picos de Europa (Picos)
- Sierra de las Nieves (SierraNieves)
- Sierra Nevada (SierraNevada)
- Teide (Teide)

## Fragstat Metrics Processed
The following 21 landscape metrics are processed:
- **ai**: Aggregation Index
- **circle_mn**: Mean Circle-related Metric  
- **contag**: Contagion Index
- **contig_mn**: Mean Contiguity Index
- **ed**: Edge Density
- **enn_mn**: Mean Euclidean Nearest Neighbor Distance
- **frac_mn**: Mean Fractal Dimension Index
- **iji**: Interspersion and Juxtaposition Index
- **lsi**: Landscape Shape Index
- **msidi**: Modified Simpson's Diversity Index
- **msiei**: Modified Simpson's Evenness Index
- **np**: Number of Patches
- **para_mn**: Mean Perimeter-Area Ratio
- **pd**: Patch Density
- **pr**: Patch Richness
- **shape_mn**: Mean Shape Index
- **shdi**: Shannon's Diversity Index
- **shei**: Shannon's Evenness Index
- **sidi**: Simpson's Diversity Index
- **siei**: Simpson's Evenness Index
- **ta**: Total Area

## Coordinate Systems
- **Source CRS**: Variable (depends on original Fragstats configuration)
- **Target CRS**: EPSG:3035 (LAEA Europe) for continental analysis

## Technical Requirements
- **Python Libraries**: `rasterio`, `gdal`, `os`, `shutil`
- **Input Format**: GeoTIFF (.tif) files from Fragstats
- **Output Format**: Reprojected GeoTIFF files
- **Resampling Method**: Nearest neighbor (preserves categorical data integrity)



In [ ]:
# =============================================================================
# SECTION 1: BATCH REPROJECTION OF FRAGSTATS LAYERS
# =============================================================================
# Generic script to reproject Fragstats landscape metrics
# Modify input and output paths according to your needs

import os
import rasterio
from rasterio.warp import calculate_default_transform, Resampling
from rasterio.enums import Resampling

# =============================================================================
# CONFIGURATION - MODIFY ACCORDING TO YOUR NEEDS
# =============================================================================

# Target coordinate reference system
# Common options: 'EPSG:25830' (UTM 30N Spain), 'EPSG:3035' (LAEA Europe), 'EPSG:4326' (WGS84 geographic)
target_crs = 'EPSG:3035'

# Study area name (for file organization)
study_area = 'Teide'

# Input and output paths - MODIFY THESE PATHS
input_folder = 'your/input/folder'        # Folder with Fragstats .tif files
output_folder = r'your\output\folder'    # Folder to save reprojected files

# =============================================================================
# DIRECTORY VALIDATION
# =============================================================================

# Verify that input folder exists
if not os.path.exists(input_folder):
    print(f"ERROR: Input folder does not exist: {input_folder}")
    print("Please verify the path and try again.")
else:
    print(f"✓ Input folder found: {input_folder}")

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    try:
        os.makedirs(output_folder)
        print(f"✓ Output folder created: {output_folder}")
    except Exception as e:
        print(f"ERROR: Could not create output folder: {str(e)}")
else:
    print(f"✓ Output folder found: {output_folder}")

# =============================================================================
# BATCH PROCESSING
# =============================================================================

# Get list of raster files in input folder
try:
    available_files = os.listdir(input_folder)
    
    # Filter only GeoTIFF files
    raster_files = [file for file in available_files 
                   if file.lower().endswith(('.tif', '.tiff'))]
    
    print(f"\nRaster files found: {len(raster_files)}")
    
    if len(raster_files) == 0:
        print("No .tif files found in input folder.")
        print("Verify that Fragstats files are in the correct directory.")
    
    # Counters for tracking
    successful = 0
    errors = 0
    
    # Process each raster file
    for file in raster_files:
        try:
            print(f"\nProcessing: {file}")
            
            # Complete input and output paths
            input_path = os.path.join(input_folder, file)
            output_name = f"reprojected_{file}"
            output_path = os.path.join(output_folder, output_name)
            
            # Open source file
            with rasterio.open(input_path) as src:
                
                # Verify that file has CRS information
                if src.crs is None:
                    print(f"  ⚠ Warning: {file} has no projection information")
                    print(f"    Assuming default projection. Verify results.")
                
                # Calculate transformation parameters
                transform, width, height = calculate_default_transform(
                    src.crs,           # Source coordinate system
                    target_crs,        # Target coordinate system
                    src.width,         # Original width in pixels
                    src.height,        # Original height in pixels
                    *src.bounds        # Spatial bounds (left, bottom, right, top)
                )
                
                # Prepare metadata for output file
                output_metadata = src.meta.copy()
                output_metadata.update({
                    'crs': target_crs,         # New projection
                    'transform': transform,     # New transformation matrix
                    'width': width,            # New dimensions
                    'height': height
                })
                
                # Create reprojected file
                with rasterio.open(output_path, 'w', **output_metadata) as dst:
                    
                    # Reproject each band (Fragstats typically has 1 band per metric)
                    for band in range(1, src.count + 1):
                        rasterio.warp.reproject(
                            source=rasterio.band(src, band),      # Source band
                            destination=rasterio.band(dst, band), # Destination band
                            src_transform=src.transform,          # Source transformation
                            src_crs=src.crs,                     # Source CRS
                            dst_transform=transform,              # Destination transformation
                            dst_crs=target_crs,                  # Destination CRS
                            resampling=Resampling.nearest        # Nearest neighbor (preserves exact values)
                        )
                
                successful += 1
                print(f"  ✓ Completed: {output_name}")
                
        except Exception as e:
            errors += 1
            print(f"  ✗ Error processing {file}: {str(e)}")
            continue
    
    # =============================================================================
    # FINAL SUMMARY
    # =============================================================================
    
    print(f"\n{'='*60}")
    print(f"PROCESSING SUMMARY")
    print(f"{'='*60}")
    print(f"Study area: {study_area}")
    print(f"Target projection: {target_crs}")
    print(f"Files processed successfully: {successful}")
    print(f"Files with errors: {errors}")
    print(f"Total files: {len(raster_files)}")
    print(f"Success rate: {(successful/max(len(raster_files),1)*100):.1f}%")
    print(f"\nReprojected files saved in: {output_folder}")
    
    if successful > 0:
        print(f"✓ Processing completed. Files are ready for further analysis.")
    
    if errors > 0:
        print(f"⚠ Review errors reported above and verify input files.")

except Exception as e:
    print(f"ERROR: Could not access input folder: {str(e)}")
    print("Verify that the path is correct and you have read permissions.")

In [ ]:
# =============================================================================
# SECTION 2: REPROJECTION WITH GDAL (ALTERNATIVE METHOD)
# =============================================================================
# Alternative method using GDAL for reprojection
# Useful when rasterio has issues or for better performance

import os
from osgeo import gdal

# =============================================================================
# CONFIGURATION - MODIFY ACCORDING TO YOUR NEEDS  
# =============================================================================

# Target coordinate reference system
# Examples: 'EPSG:25830', 'EPSG:3035', 'EPSG:4326', etc.
target_crs = 'EPSG:3035'

# Project/study area name
project_name = 'MyProject'

# Input and output paths - MODIFY THESE PATHS
input_directory = 'your/input/folder'   # Original Fragstats files
output_directory = r'your\output\folder'    # Files reprojected with GDAL


# =============================================================================
# REPROJECTION FUNCTION
# =============================================================================

def reproject_with_gdal(input_file, output_file, target_crs_code):
    """
    Reproject a raster file using GDAL Warp
    
    Parameters:
    -----------
    input_file : str
        Full path to input raster file
    output_file : str  
        Full path where to save reprojected file
    target_crs_code : str
        Target coordinate reference system (e.g., 'EPSG:25830')
    
    Returns:
    --------
    bool
        True if reprojection was successful, False if error occurred
    """
    try:
        # Open input dataset
        input_dataset = gdal.Open(input_file)
        
        if input_dataset is None:
            print(f"    ✗ Error: Could not open {os.path.basename(input_file)}")
            return False
        
        # Configure reprojection options
        warp_options = {
            'dstSRS': target_crs_code,           # Target coordinate system
            'resampleAlg': gdal.GRA_NearestNeighbour,  # Nearest neighbor
            'outputType': input_dataset.GetRasterBand(1).DataType,  # Maintain data type
        }
        
        # Execute reprojection with GDAL Warp
        result = gdal.Warp(output_file, input_dataset, **warp_options)
        
        # Clean up memory
        input_dataset = None
        result = None
        
        print(f"    ✓ Reprojected: {os.path.basename(output_file)}")
        return True
        
    except Exception as e:
        print(f"    ✗ Error reprojecting {os.path.basename(input_file)}: {str(e)}")
        return False

# =============================================================================
# DIRECTORY VALIDATION AND CONFIGURATION
# =============================================================================

print(f"Starting GDAL reprojection for: {project_name}")
print(f"Target coordinate system: {target_crs}")

# Verify input directory
if not os.path.exists(input_directory):
    print(f"\nERROR: Input directory not found:")
    print(f"  {input_directory}")
    print("\nPlease verify the path and modify the 'input_directory' variable")
else:
    print(f"\n✓ Valid input directory: {input_directory}")

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    try:
        os.makedirs(output_directory)
        print(f"✓ Output directory created: {output_directory}")
    except Exception as e:
        print(f"ERROR: Could not create output directory: {str(e)}")
        exit()
else:
    print(f"✓ Output directory exists: {output_directory}")

# =============================================================================
# BATCH PROCESSING
# =============================================================================

try:
    # Get list of files in input directory
    all_files = os.listdir(input_directory)
    
    # Filter GeoTIFF files
    tiff_files = [file for file in all_files 
                 if file.lower().endswith(('.tif', '.tiff'))]
    
    print(f"\nGeoTIFF files found: {len(tiff_files)}")
    
    if len(tiff_files) == 0:
        print("No .tif files found in input directory.")
        print("Verify that Fragstats files are in the correct location.")
    else:
        # Initialize counters
        successful = 0
        failed = 0
        
        print(f"\nStarting processing of {len(tiff_files)} files...")
        print("-" * 60)
        
        # Process each file
        for i, file in enumerate(tiff_files, 1):
            print(f"[{i}/{len(tiff_files)}] Processing: {file}")
            
            # Build complete paths
            input_path = os.path.join(input_directory, file)
            output_name = f"gdal_reproj_{file}"
            output_path = os.path.join(output_directory, output_name)
            
            # Verify that input file exists and is readable
            if os.path.exists(input_path):
                # Execute reprojection
                if reproject_with_gdal(input_path, output_path, target_crs):
                    successful += 1
                else:
                    failed += 1
            else:
                print(f"    ✗ File not found: {file}")
                failed += 1
        
        # =============================================================================
        # RESULTS SUMMARY
        # =============================================================================
        
        print("\n" + "="*60)
        print("GDAL PROCESSING SUMMARY")
        print("="*60)
        print(f"Project: {project_name}")
        print(f"Target CRS: {target_crs}")
        print(f"Files processed successfully: {successful}")
        print(f"Files with errors: {failed}")
        print(f"Total files: {len(tiff_files)}")
        print(f"Success rate: {(successful/len(tiff_files)*100):.1f}%")
        print(f"\nOutput directory: {output_directory}")
        
        if successful > 0:
            print(f"\n✓ Processing completed!")
            print(f"  Reprojected files are ready for use.")
            
        if failed > 0:
            print(f"\n⚠ Some files could not be processed.")
            print(f"  Review error messages above for more details.")

except Exception as e:
    print(f"\nCRITICAL ERROR: {str(e)}")
    print("Verify that:")
    print("- Input directory path is correct")
    print("- You have read permissions on input directory")
    print("- You have write permissions on output directory")
    print("- GDAL is correctly installed")

Starting GDAL reprojection for: MyProject
Target coordinate system: EPSG:3035

✓ Valid input directory: C:/Users/carlo/Desktop/Biophysical_variables(bio) - copia/Teide/lulc.tif_mw1/
✓ Output directory exists: C:\Users\carlo\Downloads\AAA

GeoTIFF files found: 60

Starting processing of 60 files...
------------------------------------------------------------
[1/60] Processing: circle_am.tif
    ✓ Reprojected: gdal_reproj_circle_am.tif
[2/60] Processing: circle_cv.tif
    ✓ Reprojected: gdal_reproj_circle_cv.tif
[3/60] Processing: circle_md.tif
    ✓ Reprojected: gdal_reproj_circle_md.tif
[4/60] Processing: circle_ra.tif
    ✓ Reprojected: gdal_reproj_circle_ra.tif
[5/60] Processing: circle_sd.tif
    ✓ Reprojected: gdal_reproj_circle_cv.tif
[3/60] Processing: circle_md.tif
    ✓ Reprojected: gdal_reproj_circle_md.tif
[4/60] Processing: circle_ra.tif
    ✓ Reprojected: gdal_reproj_circle_ra.tif
[5/60] Processing: circle_sd.tif
    ✓ Reprojected: gdal_reproj_circle_sd.tif
[6/60] Processin